In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Steps

1. Read a video
2. Convert video to frames
3. Extract faces from frames
4. Make predictions on faces (predict probability)
5. For each file name (i.e. video) get the average probability and infer class (0/1)
6. Create submission.csv file

# Libraries

In [5]:
!pip install '/kaggle/input/dlibpkg/dlib-19.19.0'

Processing /kaggle/input/dlibpkg/dlib-19.19.0
  Created wheel for dlib: filename=dlib-19.19.0-cp36-cp36m-linux_x86_64.whl size=3872032 sha256=bbe8fd546553b4ae9d6c86fc85c1cdeec6ebefe02200ae825c3a885cd68f5ea7
  Stored in directory: /root/.cache/pip/wheels/f3/da/18/38136f7dd8e242c1cc2236d574a50f34463b8c59aab887dd79
Successfully built dlib


In [24]:
import os
import math
import pandas as pd
import numpy as np
from glob import glob
import multiprocessing
import logging
import cv2 as cv
from tqdm import tqdm_notebook
import dlib
from tensorflow.keras import models

logging.basicConfig(format='%(name)s - %(levelname)s - %(asctime)s - %(filename)s - %(lineno)d - %(message)s', level=logging.INFO)

# Configs

In [26]:
BASE_DIR = '/kaggle/input'
OUTPUT_DIR = '/kaggle/working'
DATA_DIR = 'deepfake-detection-challenge'
VIDEOS = 'test_videos'
FRAMES = 'frames'
FACES = 'faces'
MODEL_DIR = 'inceptionv3-model'
MODEL_NAME = 'inceptionv3_facev1_augdata_17K_frames_20200327_005549.h5'

### Frames to Faces

In [33]:
def rect_to_bb(rect):
    """
       take a bounding predicted by dlib and convert it
       to the format (x, y, w, h) as we would normally do
       with OpenCV
    """
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)


def crop_faces(image):
    detector = dlib.get_frontal_face_detector()
    faces = detector(image, 1)
    cropped_faces = []
    if len(faces) > 0:
        rects, scores, idx = detector.run(image, 0)
        for rect in rects:
            if len(rects) > 0:
                (x, y, w, h) = rect_to_bb(rect)
                roi = image[(y - 50):(y + int(1.5*h)), (x - 50):(x + int(1.5*w))]
                try:
                    roi_ = cv.resize(roi, (224, 224))
                    cropped_faces.append(roi_)
                except:
                    pass
    return cropped_faces

# Prediction

In [31]:
def get_predictions(videos, model):
    cnt = 1
    predictions = []
    for video_path in videos:
        frame_pred = {}
        # setting up VideoCapture
        name = video_path.split('/')[-1]
        print(f"getting frames of: {name}")
        cap = cv.VideoCapture(video_path)
        frame_pred.update({"filename": name})
        # getting the frames
        frameRate = cap.get(5)  #to get the frame rate
        while(cap.isOpened()):
            frameId = cap.get(1)  #current frame number
            ret, frame = cap.read()
            if (ret) & (frameId % math.floor(frameRate) == 0):
                # face detection
                cropped_faces = crop_faces(frame)
                if len(cropped_faces) > 0:
                    for face in cropped_faces:
                        print("face(s) are cropped, sending to prediction")
                        face = face/255
                        face_ = np.expand_dims(face, axis=0)
                        label = model.predict_classes(face_)[0][0]
                        prob = model.predict_proba(face_)[0][0]
                        print(f"probability of fake: {prob}")
                        frame_pred.update({"label": label})
                        frame_pred.update({"prob": prob})
                        predictions.append(frame_pred)
            cap.release()
        print(f"video count so far: {cnt}")
        print("="*50)
        cnt += 1
    return predictions

In [36]:
videos_dir = os.path.join(BASE_DIR, DATA_DIR, VIDEOS)
videos = glob(os.path.join(videos_dir, '*.mp4'))
model = models.load_model(os.path.join(BASE_DIR, MODEL_DIR, MODEL_NAME))
predictions = get_predictions(videos, model)

getting frames of: ywauoonmlr.mp4
video count so far: 1
getting frames of: sfsayjgzrh.mp4
video count so far: 2
getting frames of: sngjsueuhs.mp4
video count so far: 3
getting frames of: eryjktdexi.mp4
video count so far: 4
getting frames of: nwvloufjty.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9973004460334778
video count so far: 5
getting frames of: sjkfxrlxxs.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999978542327881
video count so far: 6
getting frames of: rtpbawlmxr.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9995809197425842
video count so far: 7
getting frames of: rxdoimqble.mp4
video count so far: 8
getting frames of: ljauauuyka.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9883151054382324
video count so far: 9
getting frames of: lbfqksftuo.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9998272061347961
video count so far: 10
getting frames of: gfgcw

video count so far: 52
getting frames of: dzojiwfvba.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9930192828178406
video count so far: 53
getting frames of: rcecrgeotc.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9983671307563782
video count so far: 54
getting frames of: shnsajrsow.mp4
video count so far: 55
getting frames of: ptbnewtvon.mp4
video count so far: 56
getting frames of: hqzwudvhih.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999493360519409
video count so far: 57
getting frames of: nycmyuzpml.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9768307209014893
video count so far: 58
getting frames of: wcssbghcpc.mp4
video count so far: 59
getting frames of: llplvmcvbl.mp4
video count so far: 60
getting frames of: wclvkepakb.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9655624032020569
video count so far: 61
getting frames of: vajkicalux.mp4
face(s) are cro

probability of fake: 0.9999943971633911
video count so far: 104
getting frames of: yaxgpxhavq.mp4
video count so far: 105
getting frames of: ptbfnkajyi.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9986452460289001
video count so far: 106
getting frames of: imdmhwkkni.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9733171463012695
video count so far: 107
getting frames of: sylnrepacf.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9989959597587585
video count so far: 108
getting frames of: yxirnfyijn.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9984326958656311
video count so far: 109
getting frames of: ryxaqpfubf.mp4
video count so far: 110
getting frames of: yllztsrwjw.mp4
video count so far: 111
getting frames of: gfcycflhbo.mp4
video count so far: 112
getting frames of: knxltsvzyu.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999122619628906
video count so far: 113

probability of fake: 0.9947970509529114
video count so far: 152
getting frames of: ojsxxkalat.mp4
video count so far: 153
getting frames of: zbgssotnjm.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9982905983924866
video count so far: 154
getting frames of: uhakqelqri.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.985353946685791
video count so far: 155
getting frames of: rfwxcinshk.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999961853027344
video count so far: 156
getting frames of: syxobtuucp.mp4
video count so far: 157
getting frames of: sufvvwmbha.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999212026596069
video count so far: 158
getting frames of: kcjvhgvhpt.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9955804944038391
video count so far: 159
getting frames of: csnkohqxdv.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9980047345161438

face(s) are cropped, sending to prediction
probability of fake: 0.997632622718811
video count so far: 200
getting frames of: esjdyghhog.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9977450370788574
video count so far: 201
getting frames of: lbigytrrtr.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9895586967468262
video count so far: 202
getting frames of: ljouzjaqqe.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9895190596580505
video count so far: 203
getting frames of: gkutjglghz.mp4
video count so far: 204
getting frames of: dvwpvqdflx.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9993762373924255
video count so far: 205
getting frames of: yzuestxcbq.mp4
video count so far: 206
getting frames of: dsnxgrfdmd.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9853640198707581
video count so far: 207
getting frames of: rrrfjhugvb.mp4
face(s) are cropped, sending to predict

face(s) are cropped, sending to prediction
probability of fake: 0.9873615503311157
video count so far: 251
getting frames of: ilqwcbprqa.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9458155035972595
video count so far: 252
getting frames of: snlyjbnpgw.mp4
video count so far: 253
getting frames of: qswlzfgcgj.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.999703586101532
face(s) are cropped, sending to prediction
probability of fake: 0.9991278052330017
video count so far: 254
getting frames of: ypbtpunjvm.mp4
video count so far: 255
getting frames of: hweshqpfwe.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9995942711830139
video count so far: 256
getting frames of: nxzgekegsp.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999144077301025
video count so far: 257
getting frames of: xxzefxwyku.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.8434416055679321
video count so

face(s) are cropped, sending to prediction
probability of fake: 0.9912749528884888
video count so far: 300
getting frames of: ztyvglkcsf.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9884506464004517
video count so far: 301
getting frames of: hicjuubiau.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9998489618301392
video count so far: 302
getting frames of: voawxrmqyl.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9996622800827026
video count so far: 303
getting frames of: elackxuccp.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.96306312084198
video count so far: 304
getting frames of: pcyswtgick.mp4
video count so far: 305
getting frames of: hevcclcklc.mp4
video count so far: 306
getting frames of: ggdpclfcgk.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9926637411117554
video count so far: 307
getting frames of: hfsvqabzfq.mp4
face(s) are cropped, sending to predicti

probability of fake: 0.9997943043708801
video count so far: 349
getting frames of: ncmpqwmnzb.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9819709062576294
video count so far: 350
getting frames of: ipvwtgdlre.mp4
video count so far: 351
getting frames of: nplviymzlg.mp4
video count so far: 352
getting frames of: wfzjxzhdkj.mp4
video count so far: 353
getting frames of: xmkwsnuzyq.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9999234676361084
video count so far: 354
getting frames of: tvhjcfnqtg.mp4
video count so far: 355
getting frames of: petmyhjclt.mp4
video count so far: 356
getting frames of: dvkdfhrpph.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9998806715011597
video count so far: 357
getting frames of: qhsehzgxqj.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9998263716697693
video count so far: 358
getting frames of: ekelfsnqof.mp4
face(s) are cropped, sending to prediction
proba

face(s) are cropped, sending to prediction
probability of fake: 0.9892907738685608
video count so far: 399
getting frames of: evysmtpnrf.mp4
face(s) are cropped, sending to prediction
probability of fake: 0.9990090131759644
video count so far: 400


In [38]:
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv(os.path.join(OUTPUT_DIR, 'submission.csv'), index=False)

In [39]:
predictions_df.head()

,filename,label,prob
0,nwvloufjty.mp4,1,0.997300
1,sjkfxrlxxs.mp4,1,0.999998
2,rtpbawlmxr.mp4,1,0.999581
3,ljauauuyka.mp4,1,0.988315
4,lbfqksftuo.mp4,1,0.999827


In [ ]:
predictions_df[]